# Migration of index of TheHive and Cortex created with ES5 to ES7

In [356]:
ES_HOST='http://localhost:9200'

## List available indices

In [357]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index       uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   the_hive_15 KWvzTzq4RQOrSNCcaNstrw   5   0      30998           16     33.2mb         33.2mb
green  open   cortex_4    Tu7saVsaRwWZ1X6VHdsg4w   5   0       8531            8       13mb           13mb


If not, restore indexes.

## Setting up variables

In [359]:
THEHIVE_INDEX="the_hive_15"
CORTEX_INDEX="cortex_4"
NEW_THEHIVE_INDEX="new_the_hive_15"
NEW_CORTEX_INDEX="new_cortex_4"

## TheHive index

### Update the index ? 

Check wether you need to reindex or not.

In [360]:
curl -s ''${ES_HOST}/${THEHIVE_INDEX}'?human' | jq '.'${THEHIVE_INDEX}'.settings.index.version.created_string'

"5.6.13"


**If this start with '5.x' then proceed with next steps.** If result starts with '6.x' then, the index is ready for ES7.

### Create a new index

The First operation lies in creating a new index named `new_the_hive_15` with settings from current index `the_hive_15` (ensure to keep index version, needed for future upgrade).

In [361]:
curl -s -XPUT ''${ES_HOST}'/'${NEW_THEHIVE_INDEX}'' \
  -H 'Content-Type: application/json' \
  -d "$(curl -s http://localhost:9200/${THEHIVE_INDEX} |\
   jq '.the_hive_15|
   del(.settings.index.provided_name,
    .settings.index.creation_date,
    .settings.index.uuid,
    .settings.index.version,
    .settings.index.mapping.single_type,
    .mappings.doc._all)'
    )" | jq

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "new_the_hive_15"
}


#### Check

Check the new index is well created:

In [362]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   the_hive_15     KWvzTzq4RQOrSNCcaNstrw   5   0      30998           16     33.2mb         33.2mb
green  open   new_the_hive_15 UrrHOPuhRWSYdtmVSCMypw   5   0          0            0       631b           631b
green  open   cortex_4        Tu7saVsaRwWZ1X6VHdsg4w   5   0       8531            8       13mb           13mb


### Proceed to reindex

Next operation lies in running the reindex command in the newly created index. **This action can take some time**:

In [363]:
curl -s -XPOST -H 'Content-Type: application/json' ${ES_HOST}/_reindex -d '{
  "conflicts": "proceed",
  "source": {
    "index": "'${THEHIVE_INDEX}'"
  },
  "dest": {
    "index": "'${NEW_THEHIVE_INDEX}'"
  }
}' | jq

{
  "took": 7157,
  "timed_out": false,
  "total": 5900,
  "updated": 0,
  "created": 5900,
  "deleted": 0,
  "batches": 6,
  "version_conflicts": 0,
  "noops": 0,
  "retries": {
    "bulk": 0,
    "search": 0
  },
  "throttled_millis": 0,
  "requests_per_second": -1,
  "throttled_until_millis": 0,
  "failures": []
}


#### Check 

Run the following command, and ensure the new index is like the current one (size can vary):

In [364]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   the_hive_15     KWvzTzq4RQOrSNCcaNstrw   5   0      30998           16     33.2mb         33.2mb
green  open   new_the_hive_15 UrrHOPuhRWSYdtmVSCMypw   5   0      30998            0     26.1mb         26.1mb
green  open   cortex_4        Tu7saVsaRwWZ1X6VHdsg4w   5   0       8531            8       13mb           13mb



### Delete old indices

This is the thrilling part. 
Now the new index `new_the_hive_15` is created and similar the_hive_15,  older indexes **should be completely deleted** from the database. To delete index named `the_hive_15`, run the following command:  


Run the same command for older indexes if exist (the_hive_14, the_hive_13....). Elasticsearch 7.x cannot run with index created with Elasticsearch 5.x.



In [365]:
curl -s -XDELETE ${ES_HOST}/$THEHIVE_INDEX | jq

{
  "acknowledged": true
}


### Create an alias 

Before stopping Elasticsearch service, let’s create an alias to keep index names in the future.  

Doing so will allow TheHive 3.5.0 to find the index without updating the configuration file. 

In [366]:
curl -s -XPOST -H 'Content-Type: application/json'  ''${ES_HOST}'/_aliases' -d '{
    "actions": [
        {
            "add": {
                "index": "'${NEW_THEHIVE_INDEX}'",
                "alias": "'${THEHIVE_INDEX}'"
            }
        }
    ]
}' | jq

{
  "acknowledged": true
}


Check the alias has been well created by running the following command. The output should look like this:
```json
{
  "new_the_hive_15" : {
    "aliases" : {
      "the_hive_15" : { }
    }
  }
}
```

In [367]:
curl -s -XGET ${ES_HOST}/_alias?pretty 

{
  "cortex_4" : {
    "aliases" : { }
  },
  "new_the_hive_15" : {
    "aliases" : {
      "the_hive_15" : { }
    }
  }
}


### Check the alias is reachable and your index is ready for ES 7

In [378]:
curl -s ''${ES_HOST}/${THEHIVE_INDEX}'?human' | jq '.'${NEW_THEHIVE_INDEX}'.settings.index.version.created_string'

"6.8.0"


**TheHive is ready to be used with the restored data**

## Cortex index

### Update the index ? 

In [368]:
curl -s ''${ES_HOST}/${CORTEX_INDEX}'?human' | jq '.'${CORTEX_INDEX}'.settings.index.version.created_string'

"5.6.13"


**If this start with '5.x' then proceed with next steps.** If result starts with '6.x' then, the index is ready for ES7.

### Create a new index

The First operation lies in creating a new index named `new_cortex_4` with settings from current index `cortex_4` (ensure to keep index version, needed for future upgrade).

In [369]:
curl -s -XPUT ''${ES_HOST}/${NEW_CORTEX_INDEX}'' \
  -H 'Content-Type: application/json' \
  -d "$(curl -s http://localhost:9200/${CORTEX_INDEX} |\
   jq '.cortex_4 |
   del(.settings.index.provided_name,
    .settings.index.creation_date,
    .settings.index.uuid,
    .settings.index.version,
    .settings.index.mapping.single_type,
    .mappings.doc._all)'
    )" | jq

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "new_cortex_4"
}


#### Check

Check the new index is well created:

In [370]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_cortex_4    uOAEcbBmQ7-SOfPc1bqPDg   5   0          0            0       460b           460b
green  open   new_the_hive_15 UrrHOPuhRWSYdtmVSCMypw   5   0      30998            0     26.1mb         26.1mb
green  open   cortex_4        Tu7saVsaRwWZ1X6VHdsg4w   5   0       8531            8       13mb           13mb


### Proceed to reindex

Next operation lies in running the reindex command in the newly created index:

In [371]:
curl -s -XPOST -H 'Content-Type: application/json' ${ES_HOST}/_reindex -d '{
  "conflicts": "proceed",
  "source": {
    "index": "'${CORTEX_INDEX}'"
  },
  "dest": {
    "index": "'${NEW_CORTEX_INDEX}'"
  }
}' | jq

{
  "took": 1470,
  "timed_out": false,
  "total": 8155,
  "updated": 0,
  "created": 8155,
  "deleted": 0,
  "batches": 9,
  "version_conflicts": 0,
  "noops": 0,
  "retries": {
    "bulk": 0,
    "search": 0
  },
  "throttled_millis": 0,
  "requests_per_second": -1,
  "throttled_until_millis": 0,
  "failures": []
}


#### Check 

Run the following command, and ensure the new index is like the current one (size can vary):

In [372]:
curl -s ${ES_HOST}/_cat/indices\?v

bash: fg: %%bash: no such job
health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_cortex_4    uOAEcbBmQ7-SOfPc1bqPDg   5   0       8531            0       460b           460b
green  open   new_the_hive_15 UrrHOPuhRWSYdtmVSCMypw   5   0      30998            0     26.1mb         26.1mb
green  open   cortex_4        Tu7saVsaRwWZ1X6VHdsg4w   5   0       8531            8       13mb           13mb



### Delete old indices

This is the thrilling part. 
Now the new index `new_cortex_4` is created and similar cortex_4,  older indexes **should be completely deleted** from the database. To delete index named `cortex_4`, run the following command:  


Run the same command for older indexes if exist (cortex_1, cortex_2....). Elasticsearch 7.x cannot run with index created with Elasticsearch 5.x.



In [373]:
curl -s -XDELETE ${ES_HOST}/$CORTEX_INDEX | jq

{
  "acknowledged": true
}


### Create an alias 

Before stopping Elasticsearch service, let’s create an alias to keep index names in the future.  

Doing so will allow Cortex 3.1.0 to find the index without updating the configuration file. 

In [374]:
curl -s -XPOST -H 'Content-Type: application/json'  ''${ES_HOST}'/_aliases' -d '{
    "actions": [
        {
            "add": {
                "index": "'${NEW_CORTEX_INDEX}'",
                "alias": "'${CORTEX_INDEX}'"
            }
        }
    ]
}' | jq

{
  "acknowledged": true
}


Check the alias has been well created by running the following command:

In [375]:
curl -s -XGET ${ES_HOST}/_alias?pretty 

{
  "new_cortex_4" : {
    "aliases" : {
      "cortex_4" : { }
    }
  },
  "new_the_hive_15" : {
    "aliases" : {
      "the_hive_15" : { }
    }
  }
}


### Check the alias is reachable and version of index is 6.x

In [376]:
curl -s ''${ES_HOST}/${CORTEX_INDEX}'?human' | jq '.'${NEW_CORTEX_INDEX}'.settings.index.version.created_string'

"6.8.0"


**Cortex is ready to be used with the restored data**